In [1]:
# delivery_service.py
from flask import Flask, request, jsonify
import configparser

# config
CONFIG = "config/config.ini"
config = configparser.ConfigParser()
config.read(CONFIG)
API_KEY_OPENAI = config["API_KEYS"]["OPENAI"]

app = Flask(__name__)


@app.route("/get_delivery_estimate", methods=["POST"])
def get_delivery_estimate():
    data = request.json
    order_id = data.get("order_id")

    # Dummydata
    return jsonify(
        {
            "order_id": order_id,
            "estimated_delivery_date": "2025-08-20",
            "status": "OK",
            "warning": "Delay due to customs",
        }
    )


if __name__ == "__main__":
    app.run(port=5001)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [22/Jul/2025 17:00:33] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [22/Jul/2025 17:00:33] "GET /favicon.ico HTTP/1.1" 404 -


In [3]:
tool_definition = {
    "type": "function",
    "function": {
        "name": "get_delivery_estimate",
        "description": "Hämtar uppskattat leveransdatum för en order",
        "parameters": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "Order-ID att hämta leveransinfo för",
                }
            },
            "required": ["order_id"],
        },
    },
}

In [ ]:
from openai import OpenAI
import uuid

client = OpenAI()

# 1. Create the assistant
assistant = client.beta.assistants.create(
    name="Order Inbox Assistant",
    instructions=(
        "Du är en ERP-expert som analyserar inkommande ordermail i strukturerad form. "
        "Ge förslag på åtgärder baserat på orderns innehåll, särskilt kundkommentarer."
    ),
    model="gpt-4o",
)

# 2. Prepare structured MCP context (the parsed order)
order_mcp = {
    "type": "incoming_order_email",
    "order_id": "11223",
    "customer": "Volvo AB",
    "delivery_date": "2025-08-15",
    "line_items": [{"product": "Produkt A", "quantity": 3}],
    "parsed_comments": [
        "Kunden vill ha delad fakturering.",
        "Viktigt att detta levereras före 15 augusti.",
    ],
}

# 3. Create a new thread (conversation)
thread = client.beta.threads.create()

# 4. Send a message + attach the MCP context as `custom_data`
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    additional_messages=[
        {
            "role": "user",
            "content": "Här är en ny order att hantera.",
        }
    ],
    custom_data={"incoming_order_email": order_mcp},
)

# 5. Poll until the run completes (or use a webhook)
import time

while True:
    run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    if run_status.status in ["completed", "failed", "cancelled"]:
        break
    time.sleep(1)

# 6. Fetch the final response from the assistant
messages = client.beta.threads.messages.list(thread_id=thread.id)
for message in messages.data:
    if message.role == "assistant":
        print("\nAssistant's reply:\n")
        print(message.content[0].text.value)

In [4]:
from openai import OpenAI

client = OpenAI(api_key=API_KEY_OPENAI)

assistant = client.beta.assistants.create(
    name="Order & Delivery Assistant",
    instructions="""
    Du hjälper till att hantera inkommande ordrar.
    För varje order analyserar du informationen och använder verktyget get_delivery_estimate vid behov.
    Föreslå åtgärder utifrån status, datum och orderkommentarer.
    """,
    model="gpt-4o",
    tools=[tool_definition],
)

In [5]:
order_mcp = {
    "type": "incoming_order_email",
    "order_id": "11223",
    "customer": "Volvo AB",
    "delivery_date": "2025-08-15",
    "line_items": [{"product": "Produkt A", "quantity": 3}],
    "parsed_comments": [
        "Kunden vill ha delad fakturering.",
        "Viktigt att detta levereras före 15 augusti.",
    ],
}

thread = client.beta.threads.create()

client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Här är en ny order att hantera.",
    file_data={"incoming_order_email": order_mcp},
)

/var/folders/8d/zz072l3d0mv392k_mtqd_3tr0000gn/T/ipykernel_17922/524724502.py:13: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
/var/folders/8d/zz072l3d0mv392k_mtqd_3tr0000gn/T/ipykernel_17922/524724502.py:15: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  client.beta.threads.messages.create(


TypeError: Messages.create() got an unexpected keyword argument 'file_data'

In [ ]:
run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant.id)

In [ ]:
tool_calls = run.required_action.submit_tool_outputs.tool_calls

# Kör lokalt API-anrop till Flask-servern
import requests

api_response = requests.post(
    "http://localhost:5001/get_delivery_estimate",
    json={"order_id": tool_calls[0].function.arguments["order_id"]},
).json()

# Skicka tillbaka resultatet till assistenten
client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[{"tool_call_id": tool_calls[0].id, "output": api_response}],
)